In [37]:
# GLOBALS
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [38]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
from sklearn.cluster import KMeans

In [39]:
df = pd.read_csv('malicious-phish.csv', names=['URL', 'Classification'])
load_dotenv()

True

In [40]:
df = df.iloc[1:].reset_index(drop=True) # drops the first row which is a header from the csv

In [41]:
df.head()

,URL,Classification
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement


format df for embeddings

In [42]:
urlList = df['URL'].tolist()

# Code to generate embeddings:

In [43]:
def getEmbeddings(urls):
    api_url = "https://api.openai.com/v1/embeddings"
    headers = {
        "Authorization": "Bearer " + OPENAI_API_KEY,
        "Content-Type": "application/json",
    }
    # Splitting the list of URLs into smaller chunks to avoid payload size limits
    chunk_size = 20  # Adjust based on API limits and testing
    embeddings = []
    for i in range(0, len(urls), chunk_size):
        data = {
            "model": "text-embedding-3-large",  # use the large model for now 1,000,000 TPM atm?
            "input": urls[i:i+chunk_size]
        }
        response = requests.post(api_url, headers=headers, json=data)
        if response.status_code == 200:
            embeddings.extend(response.json()['data'])
        else:
            print("Error:", response.text)
            break
    return embeddings

In [44]:
getEmbeddings(urlList)

cost: ??? time: ???

Dimensionality reduction

In [35]:
def dimReduction(embeddings, n_components=2):
    pca = PCA(n_components=n_components)
    reduced_embeddings = pca.fit_transform(np.array(embeddings))
    return reduced_embeddings


With the dimensionality-reduced embeddings, you can apply clustering algorithms like K-Means to cluster the URLs.

In [ ]:
def cluster_embeddings(embeddings, n_clusters=5):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(embeddings)
    return kmeans.labels_

Step 4: Visualization

In [ ]:
import matplotlib.pyplot as plt

def visualize_clusters(embeddings, labels):
    plt.scatter(embeddings[:, 0], embeddings[:, 1], c=labels, cmap='viridis', s=50, alpha=0.6)
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.title('URL Clusters')
    plt.show()

Step 5: Embeddings for training models

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Assuming you have a DataFrame `df` with 'embeddings' and 'target' columns
X_train, X_test, y_train, y_test = train_test_split(df['embeddings'].tolist(), df['target'], test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)
print("Model accuracy:", model.score(X_test, y_test))